In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import math

In [2]:
original = pd.read_csv('Bahrein_2021-04-28_1900_2021-05-05_2359-AdaBoost_1_Pre_Matrix.csv')

#Columnas que nos interesan
columnas = ['Latitude', 'Longitude', 'Cell_latitude','Cell_longitude','N1_GCID','N2_GCID','N3_GCID','N4_GCID','N1_RSRP','N2_RSRP','N3_RSRP','N4_RSRP']  
df = original[columnas]


FileNotFoundError: [Errno 2] No such file or directory: 'Bahrein_2021-04-28_1900_2021-05-05_2359-AdaBoost_1_Pre_Matrix.csv'

In [ ]:
# Especificar las columnas que deseas comprobar
columnas_comprobar = ['N1_RSRP', 'N2_RSRP', 'N3_RSRP', 'N4_RSRP']  # Reemplaza con los nombres de las columnas que deseas comprobar

# Verificar en qué filas hay al menos 3 valores distintos de 0
filas_a_mover = df[(df[columnas_comprobar] != 0).sum(axis=1) <= 2]

#filas con 4 valores
filas4 = df[(df[columnas_comprobar] != 0).sum(axis=1) == 4]
filas4.to_csv("filas4.csv", index = False)
# Guardar las filas identificadas en otro dataset
df_rellenar = filas_a_mover.copy()

df_rellenar = df_rellenar.sort_values(by=['Latitude','Longitude'])
columns_to_remove = ['Cell_latitude', 'Cell_longitude']
for column in columns_to_remove:
    if column in df_rellenar.columns:
        df_rellenar = df_rellenar.drop(columns=column)
df_rellenar.to_csv("df_rellenar.csv", index = False)



In [ ]:
#Por lo que voy a hacer es con la distancia euclidiana ver que filas están más cerca y las relleno

In [ ]:

# Columnas de interés para el cálculo de la distancia
columnas_distancia = ['Latitude', 'Longitude']

# Lista de columnas GCID y RSRP a rellenar
gcid_columns = ['N2_GCID', 'N3_GCID', 'N4_GCID']
rsrp_columns = ['N2_RSRP', 'N3_RSRP', 'N4_RSRP']

# Cargar el dataset
df_rellenar = pd.read_csv('df_rellenar.csv')

# Contador de filas procesadas
contador_filas = 0
intervalo_guardado = 10000  # Guardar cada 1000 filas (puedes ajustarlo según sea necesario)

# Nombre del archivo final
archivo_final = 'df_relleno.csv'

# Borrar el archivo final si ya existe para iniciar desde cero
open(archivo_final, 'w').close()

# Iterar sobre las filas de df_rellenar
for index, fila in df_rellenar.iterrows():
    # Incrementar el contador
    contador_filas += 1
    
    # Determinar el rango de filas a considerar para la comparación
    start_index = max(0, index - 5)
    end_index = min(len(df_rellenar), index + 6)
    
    # Seleccionar las filas adyacentes y rellenar las faltantes si es necesario
    if start_index == 0:
        end_index = min(len(df_rellenar), 11)
    if end_index == len(df_rellenar):
        start_index = max(0, len(df_rellenar) - 11)
    
    filas_resto = df_rellenar.iloc[start_index:end_index].drop(index)
    
    # Extraer la fila de interés como array
    fila_array = fila[columnas_distancia].values.reshape(1, -1)
    
    # Calcular las distancias euclidianas entre la fila de df_rellenar y las filas de filas_resto
    distancias = euclidean_distances(fila_array, filas_resto[columnas_distancia].values)
    
    # Ordenar los índices de las filas por distancia, de menor a mayor
    indices_ordenados = np.argsort(distancias.flatten())
    
    # Rellenar los valores GCID y RSRP con las filas más cercanas
    valores_gcid = set()
    valores_rsrp = set()
    
    for i, (col_gcid, col_rsrp) in enumerate(zip(gcid_columns, rsrp_columns)):
        if fila[col_gcid] == 0:
            for j in indices_ordenados:
                fila_cercana = filas_resto.iloc[j]
                if (fila_cercana['N1_GCID'] != fila['N1_GCID'] and
                    fila_cercana['N1_GCID'] not in valores_gcid):
                    
                    fila[col_gcid] = fila_cercana['N1_GCID']
                    fila[col_rsrp] = fila_cercana['N1_RSRP']
                    valores_gcid.add(fila_cercana['N1_GCID'])
                    valores_rsrp.add(fila_cercana['N1_RSRP'])
                    break

    # Crear un DataFrame temporal con la fila procesada
    df_temp = pd.DataFrame(fila).T
    
    # Agregar la fila procesada al archivo final
    df_temp.to_csv(archivo_final, mode='a', header=contador_filas == 1, index=False)

    # Imprimir un mensaje y guardar el progreso cada 1000 filas procesadas
    if contador_filas % intervalo_guardado == 0:
        print(f"Procesadas {contador_filas} filas")

print("Guardado final completado.")

In [3]:
#Primero separo en dos dataframe N1 y N2
df_relleno = pd.read_csv('df_relleno_nacho_final.csv')
df_N1 = df_relleno[['Latitude','Longitude','N1_GCID', 'N1_RSRP']].copy()
df_N2 = df_relleno[['Latitude','Longitude','N2_GCID', 'N2_RSRP']].copy()
df_N3 = df_relleno[['Latitude','Longitude','N3_GCID', 'N3_RSRP']].copy()
df_N4 = df_relleno[['Latitude','Longitude','N4_GCID', 'N4_RSRP']].copy()

#Ahora las renombro y le pongo a ambas el nombre N 
df_N1.rename(columns={'N1_GCID': 'N_GCID', 'N1_RSRP': 'N_RSRP'}, inplace=True)
df_N2.rename(columns={'N2_GCID': 'N_GCID', 'N2_RSRP': 'N_RSRP'}, inplace=True)
df_N3.rename(columns={'N3_GCID': 'N_GCID', 'N3_RSRP': 'N_RSRP'}, inplace=True)
df_N4.rename(columns={'N4_GCID': 'N_GCID', 'N4_RSRP': 'N_RSRP'}, inplace=True)

#Ahora combinamos ambos

df_combinado = pd.concat([df_N1, df_N2, df_N3, df_N4], axis=0, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'df_relleno_nacho_final.csv'

In [6]:
df_combinado.shape


(5796868, 4)

In [20]:
df_combinado[['Latitude', 'Longitude']] = df_combinado[['Latitude', 'Longitude']].round(3)


In [21]:
print(df_combinado)

         Latitude  Longitude       N_GCID  N_RSRP
0          26.200     50.611  179310091.0    31.0
1          26.200     50.611  179281685.0    56.0
2          26.200     50.611  179281703.0    39.0
3          26.200     50.611  179281685.0    31.0
4          26.200     50.611  179281685.0    53.0
...           ...        ...          ...     ...
5796863    26.282     50.603  179225611.0    41.0
5796864    26.282     50.613  179365909.0    65.0
5796865    26.282     50.613  179365913.0    58.0
5796866    26.282     50.614  179234315.0    53.0
5796867    26.282     50.621  179278353.0    34.0

[5796868 rows x 4 columns]


In [25]:
# REESTRUCTURACIÓN DE VALORES


# Obtener todos los valores únicos de N_GCID
unique_gcid = df_combinado['N_GCID'].unique()
unique_gcid = sorted(unique_gcid)

# Crear un DataFrame vacío con columnas para Latitude, Longitude y cada GCID único
columns = ['Latitude', 'Longitude'] + [str(gcid) for gcid in unique_gcid]
df_reestructurado = pd.DataFrame(columns=columns)

# Iterar sobre el DataFrame df_combinado y rellenar el DataFrame reestructurado
for _, row in df_combinado.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    gcid = str(row['N_GCID'])
    rsrp = row['N_RSRP']

    # Verificar si ya existe la combinación de Latitude y Longitude en el DataFrame reestructurado
    if ((df_reestructurado['Latitude'] == lat) & (df_reestructurado['Longitude'] == lon)).any():
        # Actualizar el valor de N_RSRP para el N_GCID correspondiente
        df_reestructurado.loc[(df_reestructurado['Latitude'] == lat) & (df_reestructurado['Longitude'] == lon), gcid] = rsrp
    else:
        # Crear una nueva fila con los valores inicializados en 0
        new_row = pd.DataFrame([{col: 0 for col in columns}])
        new_row['Latitude'] = lat
        new_row['Longitude'] = lon
        new_row[gcid] = rsrp
        df_reestructurado = pd.concat([df_reestructurado, new_row], ignore_index=True)

# Guardar el DataFrame reestructurado en un nuevo archivo CSV
df_reestructurado.to_csv('df_combinado_reestructurado.csv', index=False)

# Imprimir un mensaje confirmando que se ha guardado
print("El DataFrame se ha reestructurado y guardado en 'df_combinado_reestructurado.csv'")

C:\Users\Administrador\AppData\Local\Temp\ipykernel_8336\2840618475.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_reestructurado = pd.concat([df_reestructurado, new_row], ignore_index=True)


El DataFrame se ha reestructurado y guardado en 'df_combinado_reestructurado.csv'


In [26]:
df_reestructurado = df_reestructurado.drop(columns = ['0.0'])


In [35]:
df_reestructurado.to_csv('df_combinado_reestructurado.csv', index=False)


In [1]:
# MODELO DE DEEP LEARNING



# Cargar el dataset usando pandas
data = pd.read_csv('df_combinado_reestructurado.csv')

# Separar las entradas (X) y las salidas (y)
X = data.iloc[:, 2:].values  # Las entradas son todas las columnas excepto las dos primeras
y = data.iloc[:, :2].values  # Las salidas son las dos primeras columnas

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reducir la dimensionalidad con PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=50)  # Ajusta el número de componentes principales según el rendimiento
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Definir el modelo con menos entradas y regularización
model = Sequential()
model.add(Dense(512, input_dim=X_train_pca.shape[1], activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(y_train.shape[1], activation='linear'))

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Ajustar el modelo
history = model.fit(X_train_pca, y_train, epochs=1000, batch_size=32, validation_split=0.2, verbose=0)

# Evaluar el modelo
loss = model.evaluate(X_test_pca, y_test)
print(f'Pérdida en el conjunto de prueba: {loss}')

# Hacer predicciones
predictions = model.predict(X_test_pca)

# Calcular la distancia de Haversine media
distances = []
for i in range(len(y_test)):
    real_coord = (y_test[i, 0], y_test[i, 1])
    pred_coord = (predictions[i, 0], predictions[i, 1])
    distance = haversine(real_coord, pred_coord, unit=Unit.KILOMETERS)
    distances.append(distance)

mean_distance = np.mean(distances)
print(f'Distancia de Haversine media: {mean_distance:.4f} km')



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - loss: 0.0025
Pérdida en el conjunto de prueba: 0.002560829510912299
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Distancia de Haversine media: 4.7398 km


In [2]:
# MODELO DE ARBOL ALEATORIO


# Función para calcular la distancia Haversine en kilómetros
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Cargar y preparar los datos
df = pd.read_csv('df_combinado_reestructurado.csv')  # Reemplaza 'your_data.csv' con el archivo que contiene tus datos

# Las primeras dos columnas son las salidas (latitud y longitud)
y = df.iloc[:, :2].values

# Todas las columnas restantes son las entradas
X = df.iloc[:, 2:].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las características de entrada
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir el modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Hacer predicciones con el modelo
predictions = model.predict(X_test)

# Calcular el MSE
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error on test data: {mse}')

# Calcular la distancia Haversine para cada par de coordenadas
distances_km = np.array([haversine(y_test[i, 0], y_test[i, 1], predictions[i, 0], predictions[i, 1]) for i in range(len(y_test))])

# Calcular la media de la distancia en kilómetros
mean_distance_km = np.mean(distances_km)
print(f'Mean Haversine Distance on test data in kilometers: {mean_distance_km}')


Mean Squared Error on test data: 5.602394357907047e-05
Mean Haversine Distance on test data in kilometers: 0.6033130048965964


In [3]:

df.shape

(4201, 2352)